In [1]:
#分水岭计算边界
import cv2
import numpy as np

img = cv2.imread('../images/water_coins.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
cv2.imshow('Binary Image', thresh)

# 使用形态学运算滤除噪点
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)

# 靠近目标中心的是前景 远离目标中心的是背景 硬币边缘是未知区域
# 确定背景
sure_bg = cv2.dilate(opening,kernel,iterations=3)
cv2.imshow('Background Image', sure_bg)

# 确定前景
# 使用距离转换让硬币之间分开
# 如果只是单纯抠前景 则可以不使用距离变换
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)  #离背景越远越亮，cv2.DIST_L2是指欧几里得距离（根号下平方）
ret, sure_fg = cv2.threshold(dist_transform,0.7*dist_transform.max(),255,0)  #就是从背景到最中心的位置的70%都置为0
cv2.imshow('Foreground Image', sure_fg)

# 确定未知区域
# 背景图减去前景图得到未知区域
# 类似于同心圆中大圆减去小圆得到圆环 圆环就是未知区域
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg,sure_fg)
cv2.imshow('Unknown', unknown)

# 从0开始进行标记
# 这个函数可以将各个连通域从0开始标号 同一个连通域的像素的标签相同
ret, markers = cv2.connectedComponents(sure_fg)

print(markers.shape)  #就是原图的大小，直接在图上标注出来

temp = np.uint8(markers)
temp[temp == 1] = 255

numSet = markers.reshape([-1]).tolist()
numOfCoins = len(set(numSet)) - 1
print("图中硬币数为： ", numOfCoins)
#print(len(numSet) - 1)

cv2.imshow('markers', temp)

# 因为0是未知区域 所有标签自增1
markers = markers+1

# 标记未知区域 这里unknown中的白色的环状区域为未知区域
markers[unknown==255] = 0

markers = cv2.watershed(img,markers)
img[markers == -1] = [255,0,0]
cv2.imshow('Result Image', img)

cv2.waitKey()
cv2.destroyAllWindows()

(312, 252)
图中硬币数为：  24


In [2]:
#分水岭抠图
import cv2
import numpy as np

class Segmenter(object):
   def __init__(self):
      self._mask_32S = None
      self._waterImg = None
# 将掩膜转化为CV_32S
   def setMark(self, mask):
      self._mask_32S = np.int32(mask)
# 进行分水岭操作
   def waterProcess(self, img):
      self._waterImg = cv2.watershed(img, self._mask_32S)
# 获取分割后的8位图像
   def getSegmentationImg(self):
      segmentationImg = np.uint8(self._waterImg)
      return segmentationImg
# 处理分割后图像的边界值
   def getWaterSegmentationImg(self):
      waterSegmentationImg = np.copy(self._waterImg)
      waterSegmentationImg[self._waterImg == -1] = 1
      waterSegmentationImg = np.uint8(waterSegmentationImg)
      return waterSegmentationImg
# 将分水岭算法得到的图像与源图像合并 实现抠图效果
   def mergeSegmentationImg(self, waterSegmentationImg, isWhite = False):
      _, segmentMask = cv2.threshold(waterSegmentationImg, 250, 1, cv2.THRESH_BINARY)
      segmentMask = cv2.cvtColor(segmentMask, cv2.COLOR_GRAY2BGR)
      mergeImg = cv2.multiply(img, segmentMask)
      if isWhite is True:
         mergeImg[mergeImg == 0] = 255
      return mergeImg

def getBoundingRect(img, pattern):
   _, contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
   x, y, w, h = cv2.boundingRect(contours[1])
   cv2.rectangle(pattern, (x, y), (x + w, y + h), (0, 0, 200), 2)

#主函数
img = cv2.imread('../images/Drone.jpg')
mySegmenter = Segmenter()
# 获取前景图片
grayImg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
blurImg = cv2.blur(grayImg, (3, 3))
_, binImg = cv2.threshold(blurImg, 30, 255, cv2.THRESH_BINARY_INV)
kernel1 = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
fgImg = cv2.morphologyEx(binImg, cv2.MORPH_CLOSE, kernel1)
#cv2.imshow('fgImg', fgImg)
# 获取背景图片
kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
dilateImg = cv2.dilate(binImg, kernel2, iterations=4)
_, bgImg = cv2.threshold(dilateImg, 1, 128, cv2.THRESH_BINARY_INV)
#cv2.imshow('bgImg', bgImg)
# 合成掩膜
maskImg = cv2.add(fgImg, bgImg)
mySegmenter.setMark(maskImg)
# 进行分水岭操作 并获得分割图像
mySegmenter.waterProcess(img)
waterSegmentationImg = mySegmenter.getWaterSegmentationImg()
outputImgWhite = mySegmenter.mergeSegmentationImg(waterSegmentationImg,True)
kernel3 = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 20))
dilateImg = cv2.dilate(waterSegmentationImg, kernel3)
_, dilateImg = cv2.threshold(dilateImg, 130, 255, cv2.THRESH_BINARY)
# 寻找轮廓
getBoundingRect(dilateImg, img)
cv2.imshow('Contours Image', dilateImg)
cv2.imshow('White Image', outputImgWhite)
cv2.imshow('Mask Image', maskImg)
cv2.imshow('Output Image', img)
cv2.waitKey()
cv2.destroyAllWindows()